In [1]:
# used for manipulating directory paths
import os

# Scientific and vector computation for python
import numpy as np

# Plotting library
from matplotlib import pyplot

# Optimization module in scipy
from scipy import optimize

# tells matplotlib to embed plots within the notebook
%matplotlib inline

In [2]:
# datos de entrenamiento almacenados en los arreglos X, y
data = np.loadtxt("wine_preparado.csv", delimiter=',')

# seleccionamos las columnas para X y y
X, y = data[:, 1:], data[:,0]

# cada elemento(e) del arreglo y se convierte en entero
y = np.array([int(e) for e in y])
print(y.shape)
# elimina su segunda dimesion ejemplo (n,1) se elimina 1
y = np.squeeze(y)

# reorganizamos al columna de y
y[y == 1] = 0
y[y == 2] = 1
y[y == 3] = 2

# Seleccionamos los valores de las caracteristicaa de prueba
Xp= X[150:,:]
# Seleccionamos los valores de las caracteristicaa de entrenamiento
X = X[:150,:]

# Seleccionamos los valores de y de prueba
yp = y[150:]
# Seleccionamos los valores de y de entrenamiento
y = y[:150]

# imprimimos las dimensiones de entrenamiento
print(X.shape)
print(y.shape)
# imprimimos las dimensiones de prueba
print(Xp.shape)
print(yp.shape)

# imprimimos todos los valores de y de entrenamiento
print(y)
# imprimimos todos los valores de y de prueba
print(yp)

(178,)
(150, 13)
(150,)
(28, 13)
(28,)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]


In [3]:
# Configurando parametros necesario
input_layer_size  = 13  # Entrada de 13 caracteristicas (columnas de X)
hidden_layer_size = 10   # 10 unidades ocultas ((columna de caracteristicas+1 + num_labels)/2)
num_labels = 3          # 3 etiquetas, de 0 a 2 (valores de y)

# creamos el diccionario pesos
pesos = {}
# inicializamos theta1 y theta2 creando una matriz de valores randomicos de 10x14 y 3x11 respectivamente
pesos['Theta1'] = np.random.rand(hidden_layer_size, input_layer_size+1)
pesos['Theta2'] = np.random.rand(num_labels, hidden_layer_size+1)

#asigna los valores de pesos[theta1 y 2] en theta1 y theta2
Theta1, Theta2 = pesos['Theta1'], pesos['Theta2']

# la funcion ravel transforma una matriz multidimensional en una matriz unidimensional
print(Theta1.ravel().shape)
print(Theta2.ravel().shape)

# inicializamos nn_params uniendo ambas matrices unidimenionales en una sola columna
nn_params = np.concatenate([Theta1.ravel(), Theta2.ravel()])
print(nn_params.shape)

(140,)
(33,)
(173,)


In [4]:
# funcion sigmoide
# pone cada valor del dataset en un rango de 0 a 1, como una probabilidad, por ejemplo, de cuan positiva o deseable sea esa caracteristica del vino
def sigmoid(z):

    return 1.0 / (1.0 + np.exp(-z))

# funcion del gradiente de la funcion sigmoide
# calcular los gradientes de los pesos en las capas ocultas
# ayuda a ajustar los pesos de manera que la red neuronal pueda
# aprender de manera efectiva a partir de los datos de entrenamiento
def sigmoidGradient(z):

    g = np.zeros(z.shape)

    g = sigmoid(z) * (1 - sigmoid(z))

    return g

In [5]:
def nnCostFunction(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, lambda_= 1.0):

    # Remodele nn_params nuevamente en los parámetros Theta1 y Theta2, las matrices
    # de peso para nuestra red neuronal de 2 capas.
    # Se toma una parte del vector de parámetros nn_params correspondiente a los
    # pesos entre la capa de entrada y la capa oculta.
    #Estos pesos se remodelan en una matriz Theta1 con hidden_layer_size filas y (input_layer_size + 1) columnas.
    #La dimensión (input_layer_size + 1) surge porque cada neurona en la capa oculta tiene un peso adicional
    # correspondiente al término de sesgo.
    Theta1 = np.reshape(nn_params[:hidden_layer_size * (input_layer_size + 1)],
                        (hidden_layer_size, (input_layer_size + 1)))# 10, 14
    # Se toma la parte restante del vector de parámetros nn_params correspondiente
    # a los pesos entre la capa oculta y la capa de salida.
    # Estos pesos se remodelan en una matriz Theta2 con num_labels filas (el número de etiquetas de salida)
    # y (hidden_layer_size + 1) columnas.
    #La dimensión (hidden_layer_size + 1) surge por la misma razón que en Theta1, para incluir el término
    # de sesgo en cada neurona de la capa oculta.
    Theta2 = np.reshape(nn_params[(hidden_layer_size * (input_layer_size + 1)):],
                        (num_labels, (hidden_layer_size + 1))) #3,11

    m = y.size

    J = 0
    
    # Estas líneas de código inicializan las matrices Theta1_grad y Theta2_grad con ceros. Estas matrices
    # se utilizarán para almacenar los gradientes de la función de costo con respecto a los pesos Theta1 y Theta2
    Theta1_grad = np.zeros(Theta1.shape)
    Theta2_grad = np.zeros(Theta2.shape)
    
    #capa de entrada
    #columna de 1(sesgo)
    #son parámetros adicionales en una red neuronal que permiten realizar ajustes en la salida de cada neurona
    a1 = np.concatenate([np.ones((m, 1)), X], axis=1)

    #capa oculta
    #Se calculan las activaciones de la capa oculta aplicando la función sigmoide a la combinación lineal de las
    #activaciones de la capa de entrada (a1) y los pesos de la capa oculta (Theta1).
    a2 = sigmoid(a1.dot(Theta1.T))
    a2 = np.concatenate([np.ones((a2.shape[0], 1)), a2], axis=1)

    #capa de salida
    #Se calculan las activaciones de la capa de salida aplicando la función sigmoide a la combinación lineal
    #de las activaciones de la capa oculta (a2) y los pesos de la capa de salida (Theta2).
    a3 = sigmoid(a2.dot(Theta2.T))

    #la funcion reshape convierte la matriz y en un vector unidimensional
    #se aplanará la matriz en un vector unidimensional mientras conserva el mismo número total de elementos.
    y_matrix = y.reshape(-1)

    #la funcion eye crea una matriz del tamaño de num_labels x num_labels
    y_matrix = np.eye(num_labels)[y_matrix]

    temp1 = Theta1
    temp2 = Theta2

    # Agregar el termino de regularización

    #formula de regularizacion
    #la regularizacion evita el sobreajuste, que se obtiene al ajustarse demasiado solamente a los datos de entrenamiento
    #Este término adicional penaliza los modelos que son demasiado complejos, es decir, que tienen coeficientes de pesos demasiado grandes
    reg_term = (lambda_ / (2 * m)) * (np.sum(np.square(temp1[:, 1:])) + np.sum(np.square(temp2[:, 1:])))

    #funcion de costo
    J = (-1 / m) * np.sum((np.log(a3) * y_matrix) + np.log(1 - a3) * (1 - y_matrix)) + reg_term

    # Backpropogation
    # para calcular los gradientes de la funcion de costo

    #Cálculo del error de la capa de salida
    #Se calcula el error de la capa de salida delta_3 como la diferencia entre las activaciones
    # de la capa de salida (a3) y las etiquetas reales (y_matrix)
    delta_3 = a3 - y_matrix
    #Cálculo del error de la capa oculta
    #Se calcula el error de la capa oculta delta_2 propagando hacia atrás el error de la capa de salida (delta_3
    delta_2 = delta_3.dot(Theta2)[:, 1:] * sigmoidGradient(a1.dot(Theta1.T))

    #Cálculo de los gradientes de los pesos
    #Se calculan los gradientes de los pesos de la red neuronal utilizando los errores de las capas de salida y oculta.
    Delta1 = delta_2.T.dot(a1)
    Delta2 = delta_3.T.dot(a2)

    # Agregar regularización al gradiente

    # Regularización del gradiente para Theta1
    # Primero, se calcula el gradiente de los pesos entre la capa de entrada y la capa oculta 
    Theta1_grad = (1 / m) * Delta1
    # se aplica regularización al gradiente. Para hacer esto, se ajustan los valores de los gradientes de los pesos,
    # excepto para los sesgos (es decir, para todas las columnas excepto la primera), sumando el término de regularización
    # a cada elemento.
    Theta1_grad[:, 1:] = Theta1_grad[:, 1:] + (lambda_ / m) * Theta1[:, 1:]

    # Regularización del gradiente para Theta2
    Theta2_grad = (1 / m) * Delta2
    Theta2_grad[:, 1:] = Theta2_grad[:, 1:] + (lambda_ / m) * Theta2[:, 1:]

    # Concatenación de los gradientes
    grad = np.concatenate([Theta1_grad.ravel(), Theta2_grad.ravel()])

    return J, grad

In [6]:
# En este caso, lambda_ se establece en 0, lo que significa que no se aplica regularización al 
lambda_ = 1
# El valor de J se asigna a la variable J, mientras que el valor _ se utiliza para almacenar cualquier otro
# resultado devuelto por nnCostFunction que no sea relevante para el propósito actual.
J, _ = nnCostFunction(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, lambda_)
print('Costo en parametros: %.6f ' % J)

Costo en parametros: 11.912688 


In [7]:
def randInitializeWeights(L_in, L_out, epsilon_init=0.2):
    """
    Inicializa aleatoriamente los pesos de una capa en una red neuronal.

    Parameters
    ----------
    L_in : int
        Número de conexiones entrantes.

    L_out : int
        Número de conexiones salientes.

    epsilon_init : float, optional
        Rango de valores que puede tomar el peso de un uniforme
        distribución.

    Returns
    -------
    W : array_like
        El peso se inicializó a valores aleatorios. Tenga en cuenta
        que W debe establecerse en una matriz de tamaño (L_out, 1 + L'In)
        ya que la primera columna de W maneja los términos de "sesgo"."""""

    # Se crea una matriz de ceros de tamaño (L_out, 1 + L_in), donde L_out es
    # el número de neuronas en la capa de salida y L_in es el número de neuronas en la capa de entrada.
    W = np.zeros((L_out, 1 + L_in))
    # Se sobrescribe la matriz de ceros W con valores aleatorios en el rango [0, 1)
    W = np.random.rand(L_out, 1 + L_in) * 2 * epsilon_init - epsilon_init

    return W

In [8]:
print('Inicialización de parámetros de redes neuronales...')

initial_Theta1 = randInitializeWeights(input_layer_size, hidden_layer_size)
initial_Theta2 = randInitializeWeights(hidden_layer_size, num_labels)

# Desenrrollr parametros
initial_nn_params = np.concatenate([initial_Theta1.ravel(), initial_Theta2.ravel()], axis=0)

Inicialización de parámetros de redes neuronales...


In [9]:
# Una vez que haya completado la tarea, cambie el maxiter a un valor mayor para ver cómo ayuda más capacitación.
options= {'maxiter': 5000}

#  También deberías probar diferentes valores de lambda.
lambda_ = 1

# Crear "short hand" para minimizar la función de costos.
costFunction = lambda p: nnCostFunction(p, input_layer_size,
                                        hidden_layer_size,
                                        num_labels, X, y, lambda_)

# Ahora, costFunction es una función que toma solo un argumento.
# (los parámetros de la red neuronal)
res = optimize.minimize(costFunction,
                        initial_nn_params,
                        jac=True,
                        method='TNC',
                        options=options)

# obtener la solución de la optimización
nn_params = res.x

# Obtenga Theta1 y Theta2 de nn_params
Theta1 = np.reshape(nn_params[:hidden_layer_size * (input_layer_size + 1)],
                    (hidden_layer_size, (input_layer_size + 1)))

Theta2 = np.reshape(nn_params[(hidden_layer_size * (input_layer_size + 1)):],
                    (num_labels, (hidden_layer_size + 1)))

C:\Users\hlope\AppData\Local\Temp\ipykernel_5848\1595555740.py:14: OptimizeWarning: Unknown solver options: maxiter
  res = optimize.minimize(costFunction,
C:\Users\hlope\AppData\Local\Temp\ipykernel_5848\1332050153.py:5: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-z))


In [10]:
def predict(Theta1, Theta2, X):
    """
    Predecir la etiqueta de una entrada dada una red neuronal entrenada
    Genera la etiqueta predicha de X dados los pesos entrenados de un red neuronal (Theta1, Theta2)
    """
    # Valores útiles
    m = X.shape[0]
    num_labels = Theta2.shape[0]

    # Necesitas devolver las siguientes variables correctamente
    p = np.zeros(m)
    h1 = sigmoid(np.dot(np.concatenate([np.ones((m, 1)), X], axis=1), Theta1.T))
    h2 = sigmoid(np.dot(np.concatenate([np.ones((m, 1)), h1], axis=1), Theta2.T))
    p = np.argmax(h2, axis=1)
    return p

In [11]:
pred = predict(Theta1, Theta2, X[:,:])
print(pred)
print('Training Set Accuracy: %f' % (np.mean(pred == y[:]) * 100))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
Training Set Accuracy: 99.333333


In [12]:
pred = predict(Theta1, Theta2, Xp[:,:])
print(pred)
print('Training Set Accuracy: %f' % (np.mean(pred == yp[:]) * 100))

[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
Training Set Accuracy: 100.000000
